In [1]:
from datetime import datetime as dt
import numpy as np
import pandas as pd
import utils
from functools import reduce

In [2]:
truth_df = pd.read_csv("./data/original.csv")
X = truth_df.drop('PM2.5',axis=1)
print(X)
X = X.to_numpy()
missing_result = utils.simulate_nan(X, nan_rate=0.2)
X = missing_result['X']


      AMB_TEMP  CH4    CO  NMHC    NO   NO2   NOx   PM10    RH  SO2
0         16.0  2.0  0.53  0.18 -0.30  16.0  16.0  106.0  77.0  2.6
1         16.0  2.0  0.56  0.19  0.00  23.0  23.0  106.0  77.0  3.8
2         16.0  2.0  0.56  0.20  0.10  23.0  23.0  107.0  79.0  3.6
3         16.0  2.1  0.61  0.22  1.72  19.2  20.9  108.0  80.0  3.7
4         16.0  2.1  0.67  0.19 -0.40  19.0  18.0  105.0  80.0  3.6
...        ...  ...   ...   ...   ...   ...   ...    ...   ...  ...
8779      20.0  1.9  0.30  0.09  1.30  12.0  13.0   63.0  86.0  1.7
8780      20.0  1.9  0.28  0.09  1.40  13.0  14.0   55.0  87.0  1.6
8781      20.0  2.0  0.33  0.11  1.30  14.0  16.0   50.0  89.0  1.5
8782      19.0  2.0  0.37  0.12  1.30  19.0  20.0   52.0  89.0  1.6
8783      19.0  2.0  0.40  0.16  1.80  23.0  25.0   54.0  90.0  1.8

[8784 rows x 10 columns]


In [6]:
print(X)
max_iter = 1000
eps = 1e-07

[[ 16.     2.     0.53 ... 106.    77.     2.6 ]
 [   nan   2.      nan ... 106.    77.      nan]
 [   nan   2.     0.56 ... 107.    79.     3.6 ]
 ...
 [ 20.      nan    nan ...  50.    89.      nan]
 [ 19.     2.     0.37 ...  52.    89.     1.6 ]
 [ 19.     2.      nan ...  54.    90.     1.8 ]]


In [40]:
nr, nc = X.shape
# 8784, 10 

C = np.isnan(X) == False

# Collect M_i and O_i's
# Missing, Observed, -1 if absent
one_to_nc = np.arange(1, nc + 1, step=1)
M = one_to_nc * (C == False) - 1
O = one_to_nc * C - 1

# Generate Mu_0 and Sigma_0
# Obeserved mean only
Mu = np.nanmean(X, axis=0)

# Find the rows without missing values
observed_rows = np.where(np.isnan([sum(x) for x in X ])==False)[0]
# Sample variance
S = np.cov(X[observed_rows,].T)
if np.isnan(S).any():
    S = np.diag(np.nanvar(X, axis=0))

# Start updating
Mu_tilde, S_tilde = {}, {}
X_tilde = X.copy()
iteration = 0
conv = False
while conv == False and iteration < max_iter:
    for i in range(nr):
        S_tilde[i] = np.zeros(nc**2).reshape(nc, nc)
        if set(O[i,]) != set(one_to_nc - 1):  # missing component exists
            M_i, O_i = M[i,][M[i,] != -1], O[i,][O[i,] != -1]
            S_MM = S[np.ix_(M_i, M_i)]
            S_MO = S[np.ix_(M_i, O_i)]
            S_OM = S_MO.T
            S_OO = S[np.ix_(O_i, O_i)]
            Mu_tilde[i] = Mu[np.ix_(M_i)] + S_MO @ np.linalg.inv(S_OO) @ (
                X_tilde[i, O_i] - Mu[np.ix_(O_i)]
            )
            X_tilde[i, M_i] = Mu_tilde[i]
            S_MM_O = S_MM - S_MO @ np.linalg.inv(S_OO) @ S_OM
            S_tilde[i][np.ix_(M_i, M_i)] = S_MM_O

    # Impute conditional mean given (XO, theta_old)       
    # Update theta_old to theta_new 
    Mu_new = np.mean(X_tilde, axis=0)
    S_new = np.cov(X_tilde.T, bias=1) + reduce(np.add, S_tilde.values()) / nr

    if np.linalg.norm(Mu - Mu_new) < eps and np.linalg.norm(S - S_new, ord=2) < eps:
        conv = True

    Mu = Mu_new
    S = S_new
    iteration += 1

result = {
        "mu": Mu,
        "Sigma": S,
        "X_imputed": X_tilde,
        "C": C,
        "iteration": iteration,
}


 

In [48]:
np.cov(X[observed_rows,].T)

array([[ 2.94709428e+01, -4.22783111e-01, -4.08537945e-01,
        -2.03118947e-02, -6.73957363e-01, -1.88596362e+01,
        -1.95598047e+01, -4.71962454e+01, -1.21348039e+01,
        -3.67563795e-02],
       [-4.22783111e-01,  3.07519613e-02,  1.75067494e-02,
         7.90559927e-03,  1.71675496e-01,  8.65927100e-01,
         1.03830382e+00,  2.74435284e+00,  5.60758172e-01,
         5.77169068e-02],
       [-4.08537945e-01,  1.75067494e-02,  2.70234553e-02,
         1.27866374e-02,  2.29687558e-01,  1.01257699e+00,
         1.24179265e+00,  2.92889153e+00,  2.19217957e-01,
         1.03558133e-01],
       [-2.03118947e-02,  7.90559927e-03,  1.27866374e-02,
         2.37808717e-02,  2.25395460e-01,  6.48130160e-01,
         8.74268901e-01,  7.21052648e-01,  3.40795400e-01,
         8.52808263e-02],
       [-6.73957363e-01,  1.71675496e-01,  2.29687558e-01,
         2.25395460e-01,  1.11905380e+01,  1.12891259e+01,
         2.24747527e+01,  9.18222411e+00,  3.30492148e+00,
         1.